In [1]:
import sys
import os
src_dir = os.path.join(os.getcwd(), '../src')
if src_dir not in sys.path:
    sys.path.append(src_dir)


import numpy as np
from nn_model import *
from run_nn import *
from train_nn import *
from dataloader import *
from group_transformer import *

import pandas as pd

print(os.getcwd())

# Load the training data and the test inputs
X_all = pd.read_csv('../data/X_train.csv', index_col = 0, header=[0, 1, 2]).to_numpy() # inputs of the training set
print(X_all.shape)

# add one extra dimension to the inputs
X_all = np.expand_dims(X_all, axis=-1)

y_all = pd.read_csv('../data/y_train.csv', index_col = 0).squeeze('columns') # outputs of the training set
X_test = pd.read_csv('../data/X_test.csv', index_col = 0, header=[0, 1, 2]) # inputs of the test set


from sklearn.preprocessing import StandardScaler, LabelEncoder
y_all = LabelEncoder().fit_transform(y_all)



from sklearn.model_selection import train_test_split
# create dataset
X_train, X_val, y_train, y_val = train_test_split(X_all, 
                                                  y_all, 
                                                  test_size=0.2, 
                                                  random_state=42,
                                                  shuffle=False)


from torch.utils.data import DataLoader

train_set = MusicData(X_train, y_train)
val_set = MusicData(X_val, y_val)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)

/Users/isomorphicdude/Library/Mobile Documents/com~apple~CloudDocs/MSc/SB2.2_Statistical_Machine_Learning/Practicals/Assessed/SML_pratical/notebooks
(6000, 518)
X shape: torch.Size([4800, 518, 1])
y shape: torch.Size([4800])
X shape: torch.Size([1200, 518, 1])
y shape: torch.Size([1200])


In [4]:
n_groups = 11
n_features_per_group = [84, 84, 84, 140, 7, 7, 7, 49, 7, 42, 7]
n_hidden = 64
n_classes = 8
n_encoder_layers = 1
dropout = 0.1



group_transformer = GroupedFeaturesTransformer(
    n_groups=n_groups,
    n_features_per_group=n_features_per_group,
    n_hidden=n_hidden,
    n_classes=n_classes,
    n_heads=2,
    n_encoder_layers=n_encoder_layers,
    dropout=dropout,
)


train_loss, test_loss = run_training_loop(
    num_epochs=100,
    train_loader=train_loader,
    val_loader=val_loader,
    model=group_transformer,
    optimizer=torch.optim.Adam(group_transformer.parameters(), lr=0.1),
    loss_function=torch.nn.CrossEntropyLoss(),
    device="mps",
)

INFO:root:Path not provided, saving model to models
INFO:root:Epoch: 1, train Loss: 2.2506, val Loss: 2.1141, val Accuracy: 12.09%
INFO:root:Saving model...
INFO:root:Epoch: 6, train Loss: 2.1289, val Loss: 2.1274, val Accuracy: 11.92%
INFO:root:Epoch: 11, train Loss: 2.1162, val Loss: 2.1128, val Accuracy: 12.75%
INFO:root:Saving model...
INFO:root:Epoch: 16, train Loss: 2.1185, val Loss: 2.1212, val Accuracy: 11.68%


KeyboardInterrupt: 